In [32]:
import os
import numpy as np
import matplotlib.pyplot as plt
os.environ["KERAS_BACKEND"] = "torch"
import keras

In [33]:
# https://keras.io/getting_started/intro_to_keras_for_engineers/

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [34]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [36]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - acc: 0.5451 - loss: 1.2737 - val_acc: 0.9538 - val_loss: 0.1564
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - acc: 0.9285 - loss: 0.2435 - val_acc: 0.9760 - val_loss: 0.0831
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - acc: 0.9516 - loss: 0.1577 - val_acc: 0.9792 - val_loss: 0.0676
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - acc: 0.9626 - loss: 0.1238 - val_acc: 0.9854 - val_loss: 0.0510
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - acc: 0.9686 - loss: 0.1058 - val_acc: 0.9867 - val_loss: 0.0475
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - acc: 0.9745 - loss: 0.0866 - val_acc: 0.9886 - val_loss: 0.0418
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - acc: 0.9762 - loss: 0.0811 - val_acc: 0.9877 - val_loss: 0.0374
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - acc: 0.9783 - loss: 0.0745 - val_acc: 0.9893 - val_loss: 0.0371
Epoch 9/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/

In [37]:
model.save("final_model.keras")

In [38]:
model = keras.saving.load_model("final_model.keras")

In [39]:
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
